In [1]:
!pip install -q ultralytics
!pip install -q supervision

In [2]:
pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 73.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.3/401.3 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 73.1 MB/s eta 0:00:00:00:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.22.2
    Uninstalling huggingface-hub-0.22.2:
      Successfully uninstalled huggingface-hub-0.22.2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.39.3
    Uninstalling transformers-4.39.3:
      Successfully uninstalled transformers-4.39.3
Note: you may need to restart the kernel to use updated packages.


In [3]:
!nvidia-smi

Tue May 21 18:06:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0              27W / 250W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
!pip install numpy

In [5]:
from ultralytics import YOLOWorld
import supervision as sv
import numpy as np
import cv2

In [6]:
print(np.__version__)

1.26.4


# Initializing the YOLO World Model

In [7]:
# Initialize a YOLO-World model
model = YOLOWorld("/kaggle/input/yolov8world/pytorch/test/1/yolov8x-world.pt").cuda()  # or choose yolov8m/l-world.pt
#!git clone --filter=blob:none --quiet https://github.com/ultralytics/CLIP.git /tmp/pip-req-build-i998ncen

# Video Inference

In [8]:
model.set_classes(["car","bike","truck","tuk tuk","ambulance","bus","person","potholes","lorry"])

requirements: Ultralytics requirement ['git+https://github.com/ultralytics/CLIP.git'] not found, attempting AutoUpdate...


  Running command git clone --filter=blob:none --quiet https://github.com/ultralytics/CLIP.git /tmp/pip-req-build-budro3e8


  Cloning https://github.com/ultralytics/CLIP.git to /tmp/pip-req-build-budro3e8
  Resolved https://github.com/ultralytics/CLIP.git to commit e17416a36b45d040758327936a1ea150c13fe3d1
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 8.6 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369504 sha256=c4d84fb5f86c491b333bdd15af59635e3f8f1062ff635be35a6dc616cea7676b
  Stored in directory: /tmp/pip-ephem-wheel-cache-s7bkpa4t/wheels/2e/25/92/e26f2a227599930e19a8188891e9600b6dd36721a0f8d6ef40
Successfully built clip

requirements: AutoUpdate success ✅ 17.6s, installed 1 package: ['git+https://github.com/ultralytics/CLIP.git']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



100%|███████████████████████████████████████| 338M/338M [00:05<00:00, 64.0MiB/s]


In [9]:
!python3 -m pip install numpy
import numpy as np

In [10]:
VIDEO_PATH = r"/kaggle/input/test123/Video2 - Trim.mp4"
video_info = sv.VideoInfo.from_video_path(VIDEO_PATH)

In [11]:
n = 0

In [12]:
import csv

# Define class names
class_names = ["car", "bike", "truck", "tuk tuk", "ambulance", "bus", "person", "potholes", "lorry"]

# Initialize frame data with preset columns for each class
frame_data = []

def process_frame(frame: np.ndarray, _) -> np.ndarray:
    global frame_data
    
    # Dictionary to store class counts for the current frame
    class_counts = {class_name: 0 for class_name in class_names}
    
    results = model(frame, imgsz=480)[0]
    detections = sv.Detections.from_ultralytics(results)
    box_annotator = sv.BoxAnnotator(thickness=2, text_thickness=4, text_scale=1)

    labels = [model.names[class_id] for _, _, _, class_id, _, _ in detections]
    
    # Count occurrences of each class for the current frame
    for label in labels:
        if label in class_counts:
            class_counts[label] += 1
    
    # Update frame data with class counts
    frame_info = {"Frame": len(frame_data), **class_counts}
    frame_data.append(frame_info)
    
    # Convert counts to a formatted string for display
    labels_count = [f"{label}: {class_counts[label]}" for label in class_names if class_counts[label] > 0]
    
    frame = box_annotator.annotate(scene=frame, detections=detections, labels=labels_count)
    
    return frame

# Process the video
sv.process_video(source_path=VIDEO_PATH, target_path=r"/kaggle/working/result.mp4", callback=process_frame)

# Write frame data to CSV file
csv_file_path = "/kaggle/working/frame_data.csv"
with open(csv_file_path, mode='w', newline='') as file:
    fieldnames = ["Frame"] + class_names
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    writer.writeheader()
    for data in frame_data:
        writer.writerow(data)

print("Frame data saved to:", csv_file_path)



0: 288x480 4 cars, 1 bike, 1 truck, 1 bus, 5 persons, 113.4ms
Speed: 7.6ms preprocess, 113.4ms inference, 97.2ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 4 cars, 1 bike, 1 truck, 1 bus, 5 persons, 24.4ms
Speed: 1.9ms preprocess, 24.4ms inference, 1.4ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 4 cars, 1 bike, 1 truck, 1 bus, 4 persons, 24.1ms
Speed: 1.1ms preprocess, 24.1ms inference, 1.6ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 3 cars, 1 bike, 1 truck, 1 bus, 4 persons, 24.1ms
Speed: 1.6ms preprocess, 24.1ms inference, 1.4ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 4 cars, 1 bike, 1 truck, 1 bus, 4 persons, 24.1ms
Speed: 1.3ms preprocess, 24.1ms inference, 1.5ms postprocess per image at shape (1, 3, 288, 480)



SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead


0: 288x480 4 cars, 1 bike, 1 truck, 1 bus, 4 persons, 22.7ms
Speed: 1.4ms preprocess, 22.7ms inference, 1.5ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 3 cars, 1 bike, 1 truck, 1 bus, 4 persons, 22.5ms
Speed: 1.2ms preprocess, 22.5ms inference, 1.7ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 3 cars, 1 bike, 1 truck, 1 bus, 4 persons, 22.5ms
Speed: 1.4ms preprocess, 22.5ms inference, 1.7ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 3 cars, 1 bike, 1 truck, 1 bus, 3 persons, 22.1ms
Speed: 1.2ms preprocess, 22.1ms inference, 1.4ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 3 cars, 1 bike, 1 truck, 1 bus, 3 persons, 22.1ms
Speed: 1.5ms preprocess, 22.1ms inference, 1.5ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 2 cars, 1 bike, 1 truck, 1 bus, 4 persons, 22.1ms
Speed: 1.1ms preprocess, 22.1ms inference, 1.6ms postprocess per image at shape (1, 3, 288, 480)



SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `B

0: 288x480 2 cars, 1 bike, 1 truck, 1 bus, 4 persons, 22.2ms
Speed: 1.2ms preprocess, 22.2ms inference, 1.7ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 2 cars, 1 bike, 2 buss, 4 persons, 22.0ms
Speed: 1.1ms preprocess, 22.0ms inference, 1.4ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 2 cars, 1 bike, 2 buss, 4 persons, 21.9ms
Speed: 1.5ms preprocess, 21.9ms inference, 1.6ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 2 cars, 1 bike, 2 buss, 4 persons, 22.0ms
Speed: 1.3ms preprocess, 22.0ms inference, 1.4ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 2 cars, 1 bike, 2 buss, 4 persons, 21.8ms
Speed: 1.2ms preprocess, 21.8ms inference, 1.4ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 2 cars, 1 bike, 1 bus, 4 persons, 21.9ms
Speed: 1.3ms preprocess, 21.9ms inference, 1.3ms postprocess per image at shape (1, 3, 288, 480)



...

SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `B

0: 288x480 1 bike, 1 bus, 7 persons, 21.8ms
Speed: 1.4ms preprocess, 21.8ms inference, 1.6ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 1 bike, 1 bus, 7 persons, 21.9ms
Speed: 1.3ms preprocess, 21.9ms inference, 1.4ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 1 bike, 1 bus, 7 persons, 21.9ms
Speed: 1.0ms preprocess, 21.9ms inference, 1.5ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 1 car, 1 bike, 1 truck, 2 buss, 7 persons, 21.9ms
Speed: 1.2ms preprocess, 21.9ms inference, 1.5ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 1 car, 1 bike, 1 truck, 2 buss, 7 persons, 22.1ms
Speed: 1.5ms preprocess, 22.1ms inference, 1.4ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 1 car, 1 bike, 1 truck, 2 buss, 7 persons, 22.1ms
Speed: 1.3ms preprocess, 22.1ms inference, 1.8ms postprocess per image at shape (1, 3, 288, 480)



SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `B

0: 288x480 1 car, 1 bike, 1 truck, 2 buss, 8 persons, 22.0ms
Speed: 1.4ms preprocess, 22.0ms inference, 1.5ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 1 bike, 1 truck, 2 buss, 8 persons, 21.9ms
Speed: 1.3ms preprocess, 21.9ms inference, 1.4ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 1 bike, 1 truck, 2 buss, 8 persons, 21.9ms
Speed: 1.2ms preprocess, 21.9ms inference, 1.4ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 1 car, 1 bike, 1 truck, 2 buss, 8 persons, 22.0ms
Speed: 1.3ms preprocess, 22.0ms inference, 1.5ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 1 bike, 1 truck, 2 buss, 8 persons, 22.0ms
Speed: 1.1ms preprocess, 22.0ms inference, 1.4ms postprocess per image at shape (1, 3, 288, 480)
Frame data saved to: /kaggle/working/frame_data.csv


SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead


In [13]:
import csv

# Global variable to store frame data
frame_data = []

def process_frame(frame: np.ndarray, _) -> np.ndarray:
    print(frame.shape)
    global frame_data
    
    results = model(frame, imgsz=480)[0]
    detections = sv.Detections.from_ultralytics(results)
    box_annotator = sv.BoxAnnotator(thickness=2, text_thickness=4, text_scale=1)

    labels = [f"{model.names[class_id]} {confidence:0.2f}" for _, _, confidence, class_id, _, _ in detections]
    frame = box_annotator.annotate(scene=frame, detections=detections, labels=labels)
    
    # Extract frame data
    frame_info = {
        "Frame": len(frame_data),
        "Width": frame.shape[1],
        "Height": frame.shape[0],
        "Objects": ", ".join(labels)
    }
    
    frame_data.append(frame_info)
    
    return frame

# Process the video
sv.process_video(source_path=VIDEO_PATH, target_path=r"/kaggle/working/result.mp4", callback=process_frame)



(544, 960, 3)

0: 288x480 4 cars, 1 bike, 1 truck, 1 bus, 5 persons, 22.1ms
Speed: 1.6ms preprocess, 22.1ms inference, 1.5ms postprocess per image at shape (1, 3, 288, 480)
(544, 960, 3)

0: 288x480 4 cars, 1 bike, 1 truck, 1 bus, 5 persons, 21.9ms
Speed: 1.4ms preprocess, 21.9ms inference, 1.4ms postprocess per image at shape (1, 3, 288, 480)
(544, 960, 3)

0: 288x480 4 cars, 1 bike, 1 truck, 1 bus, 4 persons, 21.9ms
Speed: 1.4ms preprocess, 21.9ms inference, 1.5ms postprocess per image at shape (1, 3, 288, 480)
(544, 960, 3)

0: 288x480 3 cars, 1 bike, 1 truck, 1 bus, 4 persons, 22.1ms
Speed: 1.5ms preprocess, 22.1ms inference, 1.5ms postprocess per image at shape (1, 3, 288, 480)
(544, 960, 3)

0: 288x480 4 cars, 1 bike, 1 truck, 1 bus, 4 persons, 22.0ms
Speed: 1.8ms preprocess, 22.0ms inference, 1.5ms postprocess per image at shape (1, 3, 288, 480)
(544, 960, 3)

0: 288x480 4 cars, 1 bike, 1 truck, 1 bus, 4 persons, 22.1ms
Speed: 1.6ms preprocess, 22.1ms inference, 1.5ms postproces

SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead


(544, 960, 3)

0: 288x480 3 cars, 1 bike, 1 truck, 1 bus, 4 persons, 22.0ms
Speed: 1.3ms preprocess, 22.0ms inference, 1.6ms postprocess per image at shape (1, 3, 288, 480)
(544, 960, 3)

0: 288x480 3 cars, 1 bike, 1 truck, 1 bus, 4 persons, 22.0ms
Speed: 1.3ms preprocess, 22.0ms inference, 1.5ms postprocess per image at shape (1, 3, 288, 480)
(544, 960, 3)

0: 288x480 3 cars, 1 bike, 1 truck, 1 bus, 3 persons, 21.9ms
Speed: 1.3ms preprocess, 21.9ms inference, 1.4ms postprocess per image at shape (1, 3, 288, 480)
(544, 960, 3)

0: 288x480 3 cars, 1 bike, 1 truck, 1 bus, 3 persons, 21.9ms
Speed: 1.1ms preprocess, 21.9ms inference, 1.4ms postprocess per image at shape (1, 3, 288, 480)
(544, 960, 3)

0: 288x480 2 cars, 1 bike, 1 truck, 1 bus, 4 persons, 21.9ms
Speed: 1.3ms preprocess, 21.9ms inference, 1.5ms postprocess per image at shape (1, 3, 288, 480)


SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `B

(544, 960, 3)

0: 288x480 2 cars, 1 bike, 1 truck, 1 bus, 4 persons, 21.9ms
Speed: 1.1ms preprocess, 21.9ms inference, 1.5ms postprocess per image at shape (1, 3, 288, 480)
(544, 960, 3)

0: 288x480 2 cars, 1 bike, 2 buss, 4 persons, 21.9ms
Speed: 1.0ms preprocess, 21.9ms inference, 1.7ms postprocess per image at shape (1, 3, 288, 480)
(544, 960, 3)

0: 288x480 2 cars, 1 bike, 2 buss, 4 persons, 22.0ms
Speed: 1.3ms preprocess, 22.0ms inference, 1.7ms postprocess per image at shape (1, 3, 288, 480)
(544, 960, 3)

0: 288x480 2 cars, 1 bike, 2 buss, 4 persons, 22.1ms
Speed: 1.5ms preprocess, 22.1ms inference, 1.4ms postprocess per image at shape (1, 3, 288, 480)
(544, 960, 3)

0: 288x480 2 cars, 1 bike, 2 buss, 4 persons, 21.9ms
Speed: 1.2ms preprocess, 21.9ms inference, 1.5ms postprocess per image at shape (1, 3, 288, 480)
(544, 960, 3)

0: 288x480 2 cars, 1 bike, 1 bus, 4 persons, 21.8ms
Speed: 1.1ms preprocess, 21.8ms inference, 1.4ms postprocess per image at shape (1, 3, 288, 480)


...

SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `B

(544, 960, 3)

0: 288x480 1 bike, 1 bus, 7 persons, 21.9ms
Speed: 1.3ms preprocess, 21.9ms inference, 1.5ms postprocess per image at shape (1, 3, 288, 480)
(544, 960, 3)

0: 288x480 1 bike, 1 bus, 7 persons, 22.2ms
Speed: 1.7ms preprocess, 22.2ms inference, 1.4ms postprocess per image at shape (1, 3, 288, 480)
(544, 960, 3)

0: 288x480 1 bike, 1 bus, 7 persons, 22.1ms
Speed: 1.6ms preprocess, 22.1ms inference, 1.6ms postprocess per image at shape (1, 3, 288, 480)
(544, 960, 3)

0: 288x480 1 car, 1 bike, 1 truck, 2 buss, 7 persons, 22.1ms
Speed: 1.5ms preprocess, 22.1ms inference, 1.4ms postprocess per image at shape (1, 3, 288, 480)
(544, 960, 3)

0: 288x480 1 car, 1 bike, 1 truck, 2 buss, 7 persons, 22.0ms
Speed: 1.5ms preprocess, 22.0ms inference, 1.6ms postprocess per image at shape (1, 3, 288, 480)
(544, 960, 3)

0: 288x480 1 car, 1 bike, 1 truck, 2 buss, 7 persons, 21.9ms
Speed: 1.2ms preprocess, 21.9ms inference, 2.1ms postprocess per image at shape (1, 3, 288, 480)


SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead


(544, 960, 3)

0: 288x480 1 car, 1 bike, 1 truck, 2 buss, 8 persons, 22.1ms
Speed: 1.3ms preprocess, 22.1ms inference, 1.8ms postprocess per image at shape (1, 3, 288, 480)
(544, 960, 3)

0: 288x480 1 bike, 1 truck, 2 buss, 8 persons, 22.0ms
Speed: 1.3ms preprocess, 22.0ms inference, 1.7ms postprocess per image at shape (1, 3, 288, 480)
(544, 960, 3)

0: 288x480 1 bike, 1 truck, 2 buss, 8 persons, 21.9ms
Speed: 1.2ms preprocess, 21.9ms inference, 1.6ms postprocess per image at shape (1, 3, 288, 480)
(544, 960, 3)

0: 288x480 1 car, 1 bike, 1 truck, 2 buss, 8 persons, 22.0ms
Speed: 1.4ms preprocess, 22.0ms inference, 1.8ms postprocess per image at shape (1, 3, 288, 480)
(544, 960, 3)

0: 288x480 1 bike, 1 truck, 2 buss, 8 persons, 22.1ms
Speed: 1.5ms preprocess, 22.1ms inference, 1.6ms postprocess per image at shape (1, 3, 288, 480)


SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `B

In [14]:
# Write frame data to CSV file
csv_file_path = "/kaggle/working/frame_data.csv"
with open(csv_file_path, mode='w', newline='') as file:
    fieldnames = ["Frame", "Width", "Height", "Objects"]
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    writer.writeheader()
    for data in frame_data:
        writer.writerow(data)

print("Frame data saved to:", csv_file_path)

Frame data saved to: /kaggle/working/frame_data.csv


# BACKEND

In [15]:
from fastapi import FastAPI, Form

In [16]:
!pip install fastapi uvicorn
!pip install python-multipart
!pip install pyngrok

In [17]:
from fastapi import FastAPI, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware
import os

app = FastAPI()

# Configure CORS to allow requests from your frontend
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Adjust this to your frontend's URL
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Directory where uploaded files will be saved
UPLOAD_DIRECTORY = "/kaggle/working/video.mp4"
os.makedirs(UPLOAD_DIRECTORY, exist_ok=True)

@app.get("/")
async def read_root():
    return {"message": "Hello, World!"}

@app.post("/upload/")
async def upload_video(file1: UploadFile = File(...)):
    #file_location = os.path.join(UPLOAD_DIRECTORY, file1.filename)
    #with open(file_location, "wb") as f:
    #    f.write(await file.read())
    #return {"info": f"file '{file.filename}' saved at '{file_location}'"}
    print(file1)


In [18]:
!ngrok config add-authtoken 2ax1IpRRcG4GZhHVmYVQbG0pcFV_6Yeg42KGpZ8J9RtVTzJwd

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                


In [ ]:
import os
from typing import Optional, Dict
import uvicorn
import nest_asyncio
from pyngrok import ngrok
# specify a port
port = 8000
ngrok_tunnel = ngrok.connect(port, bind_tls=True)

# where we can visit our fastAPI app
print('Public URL:', ngrok_tunnel.public_url)


nest_asyncio.apply()

# finally run the app
uvicorn.run(app, port=port)

Public URL: https://27b5-34-127-102-139.ngrok-free.app


INFO:     Started server process [34]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
